어떤 다수의 문장 또는 문서를 가지고 있다고 가정할 때, 이 문서들이 서로 얼마나 유사한지 비교하기 위해서는 각 문서를 문서 벡터로 변환하여야 할 것이다. 이미 구현한 패키지인 Doc2Vec이나 Sent2Vec 등을 사용하여 훈련하는 방법이 존재하지만, 가장 간단한 방법은 **문서에 존재하는 단어 벡터들의 평균을 구하는 것**이다.

이번 챕터에서는 문서 내 각 단어들을 Word2Vec을 통해 단어 벡터로 변환하고, 이들의 평균으로 문서 벡터를 구하여 선호하는 도서와 유사한 도서를 찾아주는 도서 추천 시스템을 만들어 보자.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings('ignore')

import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import re
from PIL import Image
from io import BytesIO
from nltk.tokenize import RegexpTokenizer
import nltk
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
df = pd.read_csv('/content/drive/MyDrive/book_information.csv')
print('전체 문서의 수: ', len(df))

전체 문서의 수:  2382


In [5]:
df[:5]

,Unnamed: 0,Desc,Unnamed: 0.1,author,genre,image_link,rating,title
0,0,We know that power is shifting: From West to E...,0.0,Moisés Naím,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.63,The End of Power: From Boardrooms to Battlefie...
1,1,Following the success of The Accidental Billio...,1.0,Blake J. Harris,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.94,"Console Wars: Sega, Nintendo, and the Battle t..."
2,2,How to tap the power of social software and ne...,2.0,Chris Brogan,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.78,Trust Agents: Using the Web to Build Influence...
3,3,William J. Bernstein is an American financial ...,3.0,William J. Bernstein,Business,https://i.gr-assets.com/images/S/compressed.ph...,4.20,The Four Pillars of Investing
4,4,Amazing book. And I joined Steve Jobs and many...,4.0,Akio Morita,Business,https://i.gr-assets.com/images/S/compressed.ph...,4.05,Made in Japan: Akio Morita and Sony


In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
# 데이터 전처리
def _removeNonAscii(s):
    return "".join(i for i in s if ord(i)<128)

def make_lower_case(text):
    return text.lower()

def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words('english'))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text
    
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'[a-zA-Z]+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text

df['cleaned'] = df['Desc'].apply(_removeNonAscii)
df['cleaned'] = df.cleaned.apply(make_lower_case)
df['cleaned'] = df.cleaned.apply(remove_stop_words)
df['cleaned'] = df.cleaned.apply(remove_punctuation)
df['cleaned'] = df.cleaned.apply(remove_html)

In [9]:
df['cleaned'][:5]

0    know power shifting west east north south pres...
1    following success accidental billionaires mone...
2    tap power social software networks build busin...
3    william j bernstein american financial theoris...
4    amazing book joined steve jobs many akio morit...
Name: cleaned, dtype: object

In [10]:
# 빈 값이 생긴 행이 있다면 NAN으로 변환 후 해당 행 제거
df['cleaned'].replace('', np.nan, inplace=True)
df = df[df['cleaned'].notna()]
print('전체 문서의 수: ', len(df))

전체 문서의 수:  2381


In [11]:
# 토큰화를 수행해 corpus 만들기
corpus = []
for words in df['cleaned']:
    corpus.append(words.split())

## 2. 사전 훈련된 워드 임베딩 사용하기

In [13]:
# 사전 훈련된 Word2Vec 다운
urllib.request.urlretrieve("https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz", \
                           filename="GoogleNews-vectors-negative300.bin.gz")

('GoogleNews-vectors-negative300.bin.gz',
 <http.client.HTTPMessage at 0x7f5cab6da050>)

In [14]:
word2vec_model = Word2Vec(size=300, window=5, min_count=2, workers=-1)
word2vec_model.build_vocab(corpus)
word2vec_model.intersect_word2vec_format('GoogleNews-vectors-negative300.bin.gz', lockf=1.0, binary=True)
word2vec_model.train(corpus, total_examples = word2vec_model.corpus_count, epochs=15)

(0, 0)

## 3. 단어 벡터의 평균 구하기
각 문서에 존재하는 단어들의 벡터값의 평균을 구하여 해당 문서의 벡터값을 연산

In [ ]:
def vectors(document_list):
    document_embedding_list = []
    
    # 각 문서에 대하여
    for line in document_embedding_list:
        doc2vec = None
        count = 0
        for word in line.split():
            if word in word2vec_model.wv.vocab:
                count += 1
                # 해당 문서에 있는 모든 단어들의 벡터값을 더한다.
                if doc2vec is None:
                    doc2vec = word2vec_model[word]
                else:
                    doc2vec = doc2vec + word2vec_model[word]
                    
        if doc2vec is not None:
            # 단어 벡터를 모두 더한 벡터의 값을 문서 길이로 나눈다.
            doc2vec = doc2vec / count
            document_embedding_list.append(doc2vec)
            
    # 각 문서에 대한 문서 벡터 리스트를 리턴
    return document_embedding_list            

In [ ]:
document_embedding_list = vectors(df['cleaned'])
print('문서 벡터의 수: ', len(document_embedding_list))

문서 벡터의 수:  0


## 4. 추천 시스템 구현하기

In [ ]:
# 각 문서 벡터 간의 코사인 유사도
cosine_similarities = cosine_similarity(document_embedding_list, document_embedding_list)
print('코사인 유사도 매트릭스의 크기 :',cosine_similarities.shape)

In [ ]:
# 가장 줄거리가 유사한 5개의 책을 찾아내는 함수
def recommandations(title):
    books = df[['title','image_link']]
    
    # 책의 제목을 입력하면 해당 제목의 인덱스를 리턴받아 idx에 저장
    indices = pd.Series(df.index, index = df['title']).drop_duplicates()
    idx = indices[title]
    
    # 입력된 책과 줄거리(document embedding)가 유사한 책 5개 선정
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    
    # 가장 유사한 책 5권의 인덱스
    book_indices = [i[0] for i in sim_scores]
    
    # 전체 데이터프레임에서 해당 인덱스의 행만 추출
    recommend = books.iloc[book_indices].reset_index(drop=True)
    
    fig = plt.figure(figsize=(20,30))
    
    # 데이터프레임으로부터 순차적으로 이미지를 출력
    for index, row in recommand.iterrows():
        response = requests.get(row['image_link'])
        img = Image.open(BytesIO(response.content))
        fig.add_sbplot(1,5,index+1)
        plt.imshow(img)
        plt.title(row['title'])

In [ ]:
recommendations("The Da Vinci Code")